In [1]:
import urllib.request
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy import Selector
import selenium
from selenium import webdriver
import chromedriver_binary
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import pandas as pd
import json
import re
import numpy as np

In [2]:
df_property_data = pd.DataFrame(columns = ['Title', 'Address', 'Person', 'Telefon', 'Mobil', 'Fax', 'Kaufpreis', 'Geschätzte Monatsrate', 'Zimmer', 'Wohnfläche ca.', 'Tags', 'Typ', 'Etage', 'Etage Total', 'Schlafzimmer', 'Provision für Käufer', 'Hausgeld', 'Baujahr', 'Objektzustand', 'Heizungsart', 'Energieeffizienzklasse', 'Lage'])

options = Options()
options.headless = True

#driver = webdriver.Chrome(options=options, executable_path="chromedriver.exe")

driver = webdriver.Chrome()

class Scrap_immobilienscout(scrapy.Spider):
    
    name = 'scrap_immobilienscout'
    
    def start_requests(self):
        #Replace this url with the url of your search results
        #url = 'https://www.immobilienscout24.de/Suche/de/berlin/berlin/haus-mieten?enteredFrom=one_step_search'
        #url = 'https://www.immobilienscout24.de/Suche/de/berlin/berlin/haus-kaufen?enteredFrom=one_step_search'
        url = 'https://www.immobilienscout24.de/Suche/de/sachsen-anhalt/magdeburg/wohnung-kaufen?enteredFrom=one_step_search'
        
        first_page = urllib.request.urlopen(url)
        max_pages = first_page.read()
        max_pages = max_pages.decode("utf8")
        first_page.close()


        slctr = Selector(text = max_pages)
        max_pages = slctr.xpath('//select[@aria-label="Seitenauswahl"]/option[last()]/@value').extract_first()
        max_pages = int(max_pages) if max_pages is not None else 1
        
        base_url = url.replace('enteredFrom=one_step_search', '')
        
        for page_number in range(max_pages):
            url = base_url + 'pagenumber=' + str(page_number + 1)
            yield scrapy.Request(url, callback = self.parse_properties_list)
    
    def parse_properties_list(self, response):
        
        property_urls = response.xpath('//a[contains(@class, "result-list-entry__brand-title-container")]/@href').extract()
        
        for url in property_urls:
            if 'immobilienscout24.de' not in url:
                url = 'https://www.immobilienscout24.de' + url + '#/'
                driver.get(url)
                src = driver.page_source
                self.scrap_property(src)
    
    
    def scrap_property(self, src):
        sel = Selector(text = src)
        
        title = sel.css('h1#expose-title ::text').extract_first()
        
        address = sel.xpath('//div[@class="address-block"]//span[@class="block font-nowrap print-hide"]//text()').extract_first()
        address = address.strip() if address is not None else ''
        region = sel.css('span.zip-region-and-country::text').extract_first()
        address += ' ' + region if region is not None else ''
        
        person = sel.xpath('//div[@data-qa="contactName"]//text()').extract_first()
        
        phone = sel.xpath('//script//text()').extract_first()
        phone = phone if phone is not None else ''
        telefon = re.findall('"phoneNumber":\{"contactNumber":"(\+?[0-9 ]*)"\}', phone)
        telefon = int(telefon[0].replace(' ', '')) if len(telefon) > 0 else None
        
        mobil = re.findall('"cellPhoneNumber":\{"contactNumber":"(\+?[0-9 ]*)"\}', phone)
        mobil = mobil[0] if len(mobil) > 0 else ''
        mobil = mobil.replace(' ', '')
        fax = re.findall('"faxNumber":\{"contactNumber":"(\+?[0-9 ]*)"\}', phone)
        fax = fax[0] if len(fax) > 0 else ''
        fax = fax.replace(' ', '')
        
        
        price = sel.xpath('//dd[contains(@class, "is24qa-kaufpreis")]//text()').extract_first()
        price = re.sub('[^0-9]', '', price) if price is not None else 0
        price = int(price)
        
        m_rate = sel.xpath('//span[contains(@class, "monthly-rate-result") and contains(@class, "monthly-rate-value")]//text()').extract_first()
        m_rate = re.sub('[^0-9]', '', m_rate) if m_rate is not None else 0
        m_rate = int(m_rate)
        
        rooms_count = sel.xpath('//dd[contains(@class,"is24qa-zimmer")]/text()').extract_first()
        rooms_count = int(rooms_count) if rooms_count is not None else 0
        
        liv_area = sel.xpath('//dd[contains(@class, "is24qa-wohnflaeche-ca")]/text()').extract_first()
        liv_area = re.sub('[^0-9]', '', liv_area) if liv_area is not None else 0
        liv_area = int(liv_area)
        
        tags = sel.xpath('//div[contains(@class, "criteriagroup") and contains(@class, "boolean-listing")]//span[contains(@class, "palm-hide")]/text()').extract()
        tags = list(set(tags))
        tags = [t for t in tags if t != ' ']
        tags = tags if len(tags) > 0 else ''
        
        p_type = sel.xpath('//dd[contains(@class, "is24qa-typ")]/text()').extract_first()
        
        floor = sel.xpath('//dd[contains(@class, "is24qa-etage")]//text()').extract_first()
        floor = floor if floor is not None else ''
        floor = re.findall('[0-9]+', floor)
        total_floors = int(floor[1]) if len(floor) > 1 else 0
        floor = int(floor[0]) if len(floor) > 0 else 0
        
        bedroom = sel.xpath('//dd[contains(@class, "is24qa-schlafzimmer")]//text()').extract_first()
        bedroom = re.sub('[^0-9]', '', bedroom) if bedroom is not None else 0
        bedroom = int(bedroom)
        
        allowance = sel.xpath('//dd[contains(@class, "is24qa-hausgeld")]//text()').extract_first()
        allowance = re.sub('[^0-9]', '', allowance) if allowance is not None else 0
        allowance = int(allowance)
        
        commission = sel.xpath('//dd[contains(@class, "is24qa-provision")]/text()').extract_first()
        commission = re.search('[0-9]+,?[0-9]*', commission).group() if commission is not None else ''
        commission = commission.replace(',', '.')
        commission = float(commission) if len(commission) > 0 else 0
        
        baujahr = sel.xpath('//dd[contains(@class, "is24qa-baujahr")]//text()').extract_first()
        
        objektzustand = sel.xpath('//dd[contains(@class, "is24qa-objektzustand")]//text()').extract_first()
        
        heizungsart = sel.xpath('//dd[contains(@class, "is24qa-heizungsart")]//text()').extract_first()
        
        energieeffizienzklasse = sel.xpath('//dd[contains(@class, "is24qa-energieeffizienzklasse")]//text()').extract_first()
        
        location = sel.xpath('//pre[contains(@class, "is24qa-lage")]/text()').extract_first()

        lst_row = [title, address, person, telefon, mobil, fax, price, m_rate, rooms_count, liv_area, tags, p_type, floor, total_floors, bedroom, commission, allowance, baujahr, objektzustand, heizungsart, energieeffizienzklasse, location]
        df_property_data.loc[len(df_property_data)] = lst_row
        df_property_data.to_csv('berlin_properties.csv', index=False)
        
process = CrawlerProcess()
process.crawl(Scrap_immobilienscout)
process.start()

2020-07-18 12:58:54 [scrapy.utils.log] INFO: Scrapy 2.2.0 started (bot: scrapybot)
2020-07-18 12:58:54 [scrapy.utils.log] INFO: Versions: lxml 4.5.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Apr 18 2020, 01:56:04) - [GCC 8.4.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.9.2, Platform Linux-5.3.0-62-generic-x86_64-with-Ubuntu-18.04-bionic
2020-07-18 12:58:54 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-07-18 12:58:54 [scrapy.crawler] INFO: Overridden settings:
{}
2020-07-18 12:58:54 [scrapy.extensions.telnet] INFO: Telnet Password: 12920a27eb0118ba
2020-07-18 12:58:54 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2020-07-18 12:58:54 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy

2020-07-18 12:59:00 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2020-07-18 12:59:00 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:40111/session/2d98f19938ab01a6bf819ad370456516/url {"url": "https://www.immobilienscout24.de/expose/113311322#/"}
2020-07-18 12:59:01 [urllib3.connectionpool] DEBUG: http://127.0.0.1:40111 "POST /session/2d98f19938ab01a6bf819ad370456516/url HTTP/1.1" 200 14
2020-07-18 12:59:01 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2020-07-18 12:59:01 [selenium.webdriver.remote.remote_connection] DEBUG: GET http://127.0.0.1:40111/session/2d98f19938ab01a6bf819ad370456516/source {}
2020-07-18 12:59:01 [urllib3.connectionpool] DEBUG: http://127.0.0.1:40111 "GET /session/2d98f19938ab01a6bf819ad370456516/source HTTP/1.1" 200 620791
2020-07-18 12:59:01 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2020-07-18 12:59:01 [selenium.webdriver.remote.remote_connection] DEBUG: POS

2020-07-18 12:59:08 [scrapy.core.scraper] ERROR: Spider error processing <GET https://www.immobilienscout24.de/Suche/de/sachsen-anhalt/magdeburg/wohnung-kaufen?pagenumber=6> (referer: None)
Traceback (most recent call last):
  File "/home/promit/.local/lib/python3.6/site-packages/twisted/internet/defer.py", line 654, in _runCallbacks
    current.result = callback(current.result, *args, **kw)
  File "<ipython-input-2-288ddccd905d>", line 45, in parse_properties_list
    self.scrap_property(src)
  File "<ipython-input-2-288ddccd905d>", line 82, in scrap_property
    rooms_count = int(rooms_count) if rooms_count is not None else 0
ValueError: invalid literal for int() with base 10: ' 2,5 '
2020-07-18 12:59:08 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:40111/session/2d98f19938ab01a6bf819ad370456516/url {"url": "https://www.immobilienscout24.de/expose/120678647#/"}
2020-07-18 12:59:09 [urllib3.connectionpool] DEBUG: http://127.0.0.1:40111 "POST /session/2d98f

In [28]:
df_property_data['Wohnfläche ca.']

0       60
1       60
2       40
3       47
4       68
5     4758
6    14395
7     8228
8     5214
9      928
Name: Wohnfläche ca., dtype: object

In [3]:
df_property_data.to_csv('immobilienscout_result.csv', index=False)

In [4]:
df_property_data.columns

Index(['Title', 'Address', 'Person', 'Telefon', 'Mobil', 'Fax', 'Kaufpreis',
       'Geschätzte Monatsrate', 'Zimmer', 'Wohnfläche ca.', 'Tags', 'Typ',
       'Etage', 'Etage Total', 'Schlafzimmer', 'Provision für Käufer',
       'Hausgeld', 'Baujahr', 'Objektzustand', 'Heizungsart',
       'Energieeffizienzklasse', 'Lage'],
      dtype='object')

In [5]:
df_property_data['bruttomietrendite'] = ''

In [6]:
df_property_data.columns

Index(['Title', 'Address', 'Person', 'Telefon', 'Mobil', 'Fax', 'Kaufpreis',
       'Geschätzte Monatsrate', 'Zimmer', 'Wohnfläche ca.', 'Tags', 'Typ',
       'Etage', 'Etage Total', 'Schlafzimmer', 'Provision für Käufer',
       'Hausgeld', 'Baujahr', 'Objektzustand', 'Heizungsart',
       'Energieeffizienzklasse', 'Lage', 'bruttomietrendite'],
      dtype='object')

In [7]:
df_total_price = df_property_data[['Kaufpreis', 'Provision für Käufer']]

In [8]:
df_total_price['notar'] = 0.015 * df_property_data['Kaufpreis']

2020-07-18 12:59:24 [numexpr.utils] INFO: NumExpr defaulting to 8 threads.
2020-07-18 12:59:24 [py.warnings] WARNING: /home/promit/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.



In [9]:
df_total_price['grundbuchamt'] = 0.005 * df_property_data['Kaufpreis']

2020-07-18 12:59:27 [py.warnings] WARNING: /home/promit/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.



In [10]:
df_total_price['Grunderwerbsteur'] = 0.05 * df_property_data['Kaufpreis']

2020-07-18 12:59:28 [py.warnings] WARNING: /home/promit/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.



In [11]:
df_total_price['Provision'] = df_total_price['Kaufpreis'] * df_total_price['Provision für Käufer'] * 0.01

2020-07-18 12:59:28 [py.warnings] WARNING: /home/promit/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.



In [12]:
df_total_price['total_buying_price'] = df_total_price['Kaufpreis'] + df_total_price['notar'] + df_total_price['grundbuchamt'] + df_total_price['Grunderwerbsteur'] + df_total_price['Provision']

2020-07-18 12:59:30 [py.warnings] WARNING: /home/promit/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.



In [13]:
df_total_price

,Kaufpreis,Provision für Käufer,notar,grundbuchamt,Grunderwerbsteur,Provision,total_buying_price
0,180000,0,2700,900,9000,0,192600
1,89000,3.57,1335,445,4450,3177.3,98407.3
2,62000,5,930,310,3100,3100,69440
3,96000,7.14,1440,480,4800,6854.4,109574
4,84932,6.96,1273.98,424.66,4246.6,5911.27,96788.5
5,0,0,0,0,0,0,0
6,469500,5.95,7042.5,2347.5,23475,27935.2,530300
7,0,0,0,0,0,0,0
8,149500,5.95,2242.5,747.5,7475,8895.25,168860
9,0,0,0,0,0,0,0


In [14]:
df_finance = pd.DataFrame(columns = ['eigencapital', 'interestrate', 'mortgage_payment', 'Fremdcapital'])

In [15]:
df_finance['eigencapital'] = 0.2 * df_total_price['total_buying_price']

In [16]:
df_finance['interestrate'] = (0.01 * 0.8 * df_total_price['total_buying_price'])/12

In [17]:
df_finance['mortgage_payment'] = 0.02 * 0.8 * df_total_price['total_buying_price']

In [18]:
df_finance['Fremdcapital'] = df_total_price['total_buying_price'] - df_finance['eigencapital']

In [19]:
mietindex = 11

In [20]:
df_finance['monthly_rent'] = mietindex * df_property_data['Wohnfläche ca.']

In [21]:
Nebenkosten_per_sq_m = 0.9 

In [26]:
df_property_data['Wohnfläche ca.']

0       60
1       60
2       40
3       47
4       68
5     4758
6    14395
7     8228
8     5214
9      928
Name: Wohnfläche ca., dtype: object

In [22]:
df_finance['Nebenkosten'] = Nebenkosten_per_sq_m * df_property_data['Wohnfläche ca.']

In [23]:
df_finance['cashflow'] = df_finance['monthly_rent']-df_finance['interestrate']-df_finance['mortgage_payment']-df_finance['Nebenkosten']

In [24]:
df_finance['ROI'] = (df_finance['cashflow'] * 12)/df_finance['eigencapital'] 

ZeroDivisionError: float division by zero

In [25]:
df_finance[df_finance['cashflow'] > 0]

,eigencapital,interestrate,mortgage_payment,Fremdcapital,monthly_rent,Nebenkosten,cashflow
5,0,0,0,0,52338,4282.2,48055.8
6,106060,353.534,8484.8,424240,158345,12955.5,136551
7,0,0,0,0,90508,7405.2,83102.8
8,33772.1,112.574,2701.76,135088,57354,4692.6,49847.1
9,0,0,0,0,10208,835.2,9372.8
